In [ ]:
!pip install -q condacolab
import condacolab
condacolab.install()

In [ ]:
!pip install -qqq hdbscan biopython screed tqdm py3dmol markov-clustering pdb-tools altair vega_datasets networkx numpy pandas matplotlib pytest libpysal spreg esda geopandas
!pip install -qqq -i https://test.pypi.org/simple/ faltwerk==0.2b0

!conda config --add channels defaults
!conda config --add channels bioconda
!conda config --add channels conda-forge

!mamba install -q -y -c bioconda foldseek=3.915ef7d hmmer

In [ ]:
!pip install -i https://test.pypi.org/simple/ faltwerk==0.2rc0
!git clone https://github.com/phiweger/faltwerk
%cd faltwerk

In [ ]:
from hdbscan import HDBSCAN

from faltwerk.models import Fold
from faltwerk.vis import Layout

fp = 'data/alphafold2/transferrin/test_08df6_unrelaxed_rank_1_model_3.pdb'
model = Fold(fp)
Layout(model).geom_ribbon().render().show()

In [ ]:
import json
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

import altair as alt
import numpy as np
import py3Dmol
import pandas as pd

from libpysal.weights import KNN, DistanceBand
from spreg import OLS, Probit
# https://github.com/pysal/spreg/blob/d464bbbc3c8601f1ca1989f4756967dca3a83179/spreg/probit.py#L704

from faltwerk.biochem import solvent_access
from faltwerk.geometry import is_close, get_complex_interface, distance_to_positions, get_alpha_carbon_atoms
from faltwerk.io import load_conserved, load_bfactor_column, load_conserved
from faltwerk.models import Fold, Complex, Binding, AlphaFold
from faltwerk.stats import find_hotspots, cluster
from faltwerk.vis import Layout, plot_alphafold
from faltwerk.utils import flatten


In [ ]:
fp = 'data/alphafold2/transferrin/'
af = AlphaFold(fp)

In [ ]:
plot_alphafold(af)


In [ ]:
model = af.best

# What annotation tracks are present?
model.annotation.keys()
# dict_keys(['position', 'plddt'])

Layout(model, panel_size=(400, 300)).geom_ribbon('plddt', palette='rainbow_r').render().show()
# Blue is good
# Pick any color or color palette from matplotlib:
# https://matplotlib.org/stable/tutorials/colors/colormaps.html

In [18]:
!wget http://ftp.ebi.ac.uk/pub/databases/Pfam/releases/Pfam31.0/Pfam-A.hmm.dat.gz
!wget http://ftp.ebi.ac.uk/pub/databases/Pfam/releases/Pfam31.0/Pfam-A.hmm.gz
!gunzip -q Pfam*
!hmmpress Pfam-A.hmm

--2022-08-19 09:09:58--  http://ftp.ebi.ac.uk/pub/databases/Pfam/releases/Pfam31.0/Pfam-A.hmm.dat.gz
Resolving ftp.ebi.ac.uk (ftp.ebi.ac.uk)... 193.62.193.138
Connecting to ftp.ebi.ac.uk (ftp.ebi.ac.uk)|193.62.193.138|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 444686 (434K) [application/x-gzip]
Saving to: ‘Pfam-A.hmm.dat.gz’

Pfam-A.hmm.dat.gz   100%[===================>] 434.26K  1.30MB/s    in 0.3s    

2022-08-19 09:09:59 (1.30 MB/s) - ‘Pfam-A.hmm.dat.gz’ saved [444686/444686]

--2022-08-19 09:09:59--  http://ftp.ebi.ac.uk/pub/databases/Pfam/releases/Pfam31.0/Pfam-A.hmm.gz
Resolving ftp.ebi.ac.uk (ftp.ebi.ac.uk)... 193.62.193.138
Connecting to ftp.ebi.ac.uk (ftp.ebi.ac.uk)|193.62.193.138|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 257664436 (246M) [application/x-gzip]
Saving to: ‘Pfam-A.hmm.gz’

Pfam-A.hmm.gz       100%[===================>] 245.73M  33.1MB/s    in 10s     

2022-08-19 09:10:10 (23.9 MB/s) - ‘Pfam-A.hmm.gz’ 

In [24]:
!gunzip -h

Usage: gzip [OPTION]... [FILE]...
Compress or uncompress FILEs (by default, compress FILES in-place).

Mandatory arguments to long options are mandatory for short options too.

  -c, --stdout      write on standard output, keep original files unchanged
  -d, --decompress  decompress
  -f, --force       force overwrite of output file and compress links
  -h, --help        give this help
  -k, --keep        keep (don't delete) input files
  -l, --list        list compressed file contents
  -L, --license     display software license
  -n, --no-name     do not save or restore the original name and time stamp
  -N, --name        save or restore the original name and time stamp
  -q, --quiet       suppress all warnings
  -r, --recursive   operate recursively on directories
  -S, --suffix=SUF  use suffix SUF on compressed files
  -t, --test        test compressed file integrity
  -v, --verbose     verbose mode
  -V, --version     display version number
  -1, --fast        compress faster
  -9

In [20]:
!hmmpress Pfam-A.hmm

Working...    done.
Pressed and indexed 16712 HMMs (16712 names and 16712 accessions).
Models pressed into binary file:   Pfam-A.hmm.h3m
SSI index for binary model file:   Pfam-A.hmm.h3i
Profiles (MSV part) pressed into:  Pfam-A.hmm.h3f
Profiles (remainder) pressed into: Pfam-A.hmm.h3p
